In [1]:
def refineDataSets(data, isTest):
    
    import pandas as pd
    
    data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare'], axis=1)

    for i in range(0, data.shape[0]):
    
        if(data['Sex'][i] == 'female'):
            data['Sex'][i] = 1
        
        else:
            data['Sex'][i] = 0
    
        if(data['Embarked'][i] == 'S'):
            data['Embarked'][i] = 0
    
        elif(data['Embarked'][i] == 'C'):
            data['Embarked'][i] = 1
    
        else:
            data['Embarked'][i] = 2
    
    
    data['Embarked'][0] = 0

    import math

    for i in range(0, data.shape[0]):
        if(math.isnan(data['Age'][i])):
            data['Age'][i] = data['Age'].mean()
            
    import numpy as np
    
    x_data = np.array([data['Pclass'].values, data['Sex'].values, data['Age'].values, data['SibSp'].values, data['Parch'].values, data['Embarked'].values])  
    x_data = np.float32(np.transpose(x_data))

    if isTest:
        return x_data
    
    p_data = np.array([data['Survived'].values])
    p_data = np.float32(np.transpose(p_data))
    return x_data, p_data

In [2]:
def createAndTrainModelAndPredictSurvival(input_data):
    
    import tensorflow as tf
    import numpy as np
    import pandas as pd
    import math
    
    alpha = 0.01
    beta = 0.01

    sess = tf.InteractiveSession()
    
    train_data = pd.read_csv('/Users/krishnapaparaju/Downloads/train.csv')
    
    x_train_data, p_train_data = refineDataSets(train_data, False)
    x_input_data = refineDataSets(input_data, True)
    
    from edward.models import Normal

    sess = tf.InteractiveSession()

    W = Normal(loc=tf.zeros([6, 1]), scale=tf.ones([6, 1]))
    b = Normal(loc=tf.zeros(1), scale=tf.ones(1))

    x = tf.placeholder(tf.float32, [None, 6])
    p = Normal(loc=tf.matmul(x, W) + b,
           scale=0.1)

    qW = Normal(loc=tf.Variable(tf.zeros([6, 1])),
              scale=tf.nn.softplus(tf.Variable(tf.zeros([6, 1]))))
    qb = Normal(loc=tf.Variable(tf.zeros(1)),
              scale=tf.nn.softplus(tf.Variable(tf.zeros(1))))
    

    import edward as ed

    inference = ed.KLqp({W: qW, b: qb}, data={x: x_train_data, p: p_train_data})
    
    inference.run(n_iter=500)

    p_ = ed.copy(p, ({W: qW, b: qb})) 
    
    return sess.run(p_, feed_dict = {x: x_input_data})

In [5]:
import pandas as pd
import numpy as np

test_data = pd.read_csv('/Users/krishnapaparaju/Downloads/test.csv')
predictions = createAndTrainModelAndPredictSurvival(test_data)
np.round(predictions)

/Users/krishnapaparaju/miniconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/Users/krishnapaparaju/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/krishnapaparaju/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

ImportError: cannot import name 'set_shapes_for_outputs'